In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy.sparse as sp
from scipy.stats import binned_statistic
import matplotlib.pylab as plt
%matplotlib inline
from rubin_sim.utils import ddf_locations

In [9]:
# 100 potential time slots
times = np.arange(100)
m = gp.Model("try_sched")

schedule_1 = m.addMVar(times.size, vtype=GRB.BINARY, name="schedule_1")
schedule_2 = m.addMVar(times.size, vtype=GRB.BINARY, name="schedule_2")

# Can't be at the same time
m.addConstr(schedule_1 @ schedule_2 == 0)

mask1 = np.zeros(times.size)
mask2 = np.zeros(times.size)

# Set some random times that can't be scheduled for each
np.random.seed(42)
mask1[np.round(np.random.uniform(low=0,high=99, size=20)).astype(int)] = 1
mask2[np.round(np.random.uniform(low=0,high=99, size=20)).astype(int)] = 1

m.addConstr(schedule_1 @ mask1 == 0)
m.addConstr(schedule_2 @ mask2 == 0)

# Limit the total number of times each one can be scheduled
n_limit_1 = 30
n_limit_2 = 30
m.addConstr(schedule_1.sum() == n_limit_1)
m.addConstr(schedule_2.sum() == n_limit_2)

# I want both to be as evenly distributed as possible. So, minimize chi^2
# for the difference between the cumulative distribution and an ideal cumulative
desired_cumulative_1 = np.round(times/times.max() * n_limit_1)
desired_cumulative_2 = np.round(times/times.max() * n_limit_2)

cumulative_sched_1 = m.addMVar(times.size, vtype=GRB.CONTINUOUS)
cumulative_diff_1 = m.addMVar(times.size, vtype=GRB.CONTINUOUS, lb=-n_limit_1, ub=n_limit_1)

cumulative_sched_2 = m.addMVar(times.size, vtype=GRB.CONTINUOUS)
cumulative_diff_2 = m.addMVar(times.size, vtype=GRB.CONTINUOUS, lb=-n_limit_2, ub=n_limit_2)

m.addConstr(cumulative_sched_1[0] == schedule_1[0])
m.addConstr(cumulative_sched_2[0] == schedule_2[0])

for i in np.arange(1,times.size):
    m.addConstr(cumulative_sched_1[i] == cumulative_sched_1[i-1]+schedule_1[i])
    m.addConstr(cumulative_diff_1[i] == cumulative_sched_1[i] - desired_cumulative_1[i])
   
    m.addConstr(cumulative_sched_2[i] == cumulative_sched_2[i-1]+schedule_2[i])
    m.addConstr(cumulative_diff_2[i] == cumulative_sched_2[i] - desired_cumulative_2[i])
   



In [10]:
m.setObjective(cumulative_diff_2@cumulative_diff_2, GRB.MINIMIZE)

In [11]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 402 rows, 600 columns and 1232 nonzeros
Model fingerprint: 0x27b486fa
Model has 100 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 400 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 3e+01]
Presolve removed 208 rows and 327 columns
Presolve time: 0.01s
Presolved: 194 rows, 273 columns, 679 nonzeros
Presolved model has 99 quadratic objective terms
Variable types: 0 continuous, 273 integer (149 binary)
Found heuristic solution: objective 4190.0000000
Found heuristic solution: objective 456.0000000

Root relaxation: objective 3.000000e+00, 541 iterations, 0.01 seconds

    Nodes    |    Current Node    | 

In [12]:
schedule_1.X

array([1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1.,
       1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1.])

In [13]:
schedule_2.X

array([0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.])

In [16]:
np.cumsum(schedule_2.X) - desired_cumulative_2

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])